In [1]:
import pandas as pd
import numpy as np 

In [2]:
df = pd.read_csv('Columbia_Hackathon_Data_Dogfood.csv')

df.columns

Index(['sale_id', 'sale_date', 'ad_exp', 'sns', 'product_id', 'product_brand',
       'product_name', 'price', 'qty', 'customer_id', 'gender', 'city', 'st',
       'zip', 'lat', 'lng', 'marital', 'education', 'income', 'age', 'prime'],
      dtype='object')

In [3]:
df["total_amount"] = df["price"] * df["qty"]
# features = ['ad_exp', 'gender','marital', 'education', 'income', 'age', 'prime'] # didn't include state (too many level) and sns (negative coefficient)
features = ['ad_exp', 'income', 'age']

## 1. Lasso Regression for Feature Selection

In [ ]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.linear_model import Lasso

# X = df[features]
# X = pd.get_dummies(X)
# y = df["total_amount"]

In [162]:
# # Split the data (Don't think we need to)
# #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Create pipeline
# pipeline = Pipeline([
#                      ('scaler',StandardScaler()),
#                      ('model',Lasso())
# ])

# # Grid search
# search = GridSearchCV(pipeline,
#                       {'model__alpha':np.arange(0.1,10,0.1)},
#                       cv = 5, scoring="neg_mean_squared_error",verbose=0
#                       )
# # Fit the grid search
# search.fit(X,y)
# print("Best parameter:", search.best_params_)

# # Extract coefficients
# coefficients = search.best_estimator_.named_steps['model'].coef_
# # Convert to absolute numbers
# importance = np.abs(coefficients)
# # Significant features
# np.array(X.columns)[importance > 0]

## 2. Weights

### 2.1 Linear Regression

In [4]:
# One-hot encoder
X = df[features]
X = pd.get_dummies(X, drop_first = True)
y = df["total_amount"]

# Fit linear regression from one-hot encoder
from sklearn.linear_model import LinearRegression
reg = LinearRegression(fit_intercept = False).fit(X, y)
coef = pd.DataFrame(X.columns)
coef["coef"] = reg.coef_
coef.columns = ["features","coef"]

# Create feature group
def feature_group(text):
    result = text
    for feature in features:
        if feature in text:
            result = feature
            break
    return result
coef["features_group"] = coef["features"].apply(feature_group)
coef = coef[["features_group","features","coef"]]
coef

,features_group,features,coef
0,ad_exp,ad_exp_Don't recall seeing an ad,11.220384
1,ad_exp,ad_exp_Some other type of ad,20.261414
2,ad_exp,ad_exp_Sponsored Brands,11.738680
3,ad_exp,ad_exp_Sponsored Products,11.596665
4,ad_exp,ad_exp_Video ad,13.628577
5,income,"income_$20,000 - $39,999",6.032076
6,income,"income_$40,000 - $59,999",5.755871
7,income,"income_$60,000 - $79,999",4.088945
8,income,"income_$80,000 - $99,999",4.398496
9,income,"income_Less than $20,000",3.990224


In [5]:
# Average coeeficient and normalize
weight = pd.DataFrame(coef.groupby("features_group").mean() / coef.groupby("features_group").mean().sum()).reset_index()
weight


,features_group,coef
0,ad_exp,0.28816
1,age,0.60968
2,income,0.10216


### 2.2 Decision Tree

In [6]:
# # Decision tree
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state=123)

# # Grid search
# from sklearn.model_selection import train_test_split, GridSearchCV
# tree_para = {'criterion':["squared_error", "friedman_mse", "absolute_error", "poisson"],
#              'max_depth':[2,4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150],
#              'min_samples_split':[2,4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]
#             }

# search = GridSearchCV(regressor,
#                       tree_para,
#                       cv = 5, 
#                       scoring="neg_root_mean_squared_error",
#                       verbose=0
#                       )
# # Fit the grid search
# search.fit(X,y)
# print("Best parameter:", search.best_params_)

In [7]:
# Fit the model with best parameters
# regressor = DecisionTreeRegressor(random_state=123,
#                                 criterion = search.best_params_['criterion'],
#                                  max_depth = search.best_params_['max_depth'],
#                                  min_samples_split = search.best_params_['min_samples_split'])

regressor = DecisionTreeRegressor(random_state=123,
                                criterion = 'poisson',
                                 max_depth = 4,
                                 min_samples_split = 90)
regressor.fit(X,y)

# Extract Feature Importance
weight = pd.DataFrame({"features":regressor.feature_names_in_,
             "importance":regressor.feature_importances_})
weight["features_group"] = weight["features"].apply(feature_group)
weight = weight[["features_group","features","importance"]]
weight



,features_group,features,importance
0,ad_exp,ad_exp_Don't recall seeing an ad,0.026885
1,ad_exp,ad_exp_Some other type of ad,0.058081
2,ad_exp,ad_exp_Sponsored Brands,0.000000
3,ad_exp,ad_exp_Sponsored Products,0.027250
4,ad_exp,ad_exp_Video ad,0.179834
5,income,"income_$20,000 - $39,999",0.056094
6,income,"income_$40,000 - $59,999",0.033600
7,income,"income_$60,000 - $79,999",0.000000
8,income,"income_$80,000 - $99,999",0.065654
9,income,"income_Less than $20,000",0.108798


In [8]:
# Sum by feature group
weight = weight.groupby("features_group")["importance"].sum().reset_index()
weight = weight[weight["importance"] > 0].reset_index(drop = True)
weight

,features_group,importance
0,ad_exp,0.292050
1,age,0.443804
2,income,0.264146


In [9]:
# Change to dict
weight_dict = dict()
for i in range(len(weight)):
    weight_dict[weight.loc[i,"features_group"]] = weight.loc[i,"importance"]

weight_dict

{'ad_exp': 0.29205024407666136,
 'age': 0.44380397014767203,
 'income': 0.2641457857756668}

## 3. Scores

In [10]:
# Calculate feature score for each brand
#brands = ["Alpha", "Bezt", "Arf", "Health One", "Choice"]
brands = list(df["product_brand"].unique())
features = ["ad_exp","age","income"]

brand_score = dict()
for brand in brands:
    # Run a linear regression for each brand
    df_brand = df[df["product_brand"] == brand]
    X = df_brand[features]
    X = pd.get_dummies(X, drop_first = True)
    y = df_brand["total_amount"]
    reg = LinearRegression(fit_intercept = False).fit(X, y)
    coef = pd.DataFrame(X.columns)
    coef["coef"] = reg.coef_
    coef.columns = ["features","coef"]
    coef["features_group"] = coef["features"].apply(feature_group)
    coef = coef[["features_group","features","coef"]]
    
    # Extract coefficients and scale
    feature_dict = dict()
    for feature in features:
        if feature in ["prime","sns"]:
            value_dict = dict()
            binary_coef = coef[coef["features_group"] == "prime"].iloc[:,-1][0]
            if binary_coef > 0:
                value_dict[1] = 1
                value_dict[0] = 0
            else:
                value_dict[1] = 0
                value_dict[0] = 1

        else:
            select_coef = coef[coef["features_group"] == feature].iloc[:,1:]
            dummy_level = [] 
            for level in select_coef["features"].to_list():
                dummy_level.append(level.replace(feature+"_",""))
            all_value = df[feature].unique()
            dropped = [w for w in all_value if w not in dummy_level][0]
            all_coef = pd.DataFrame({"level":dummy_level + [dropped],
                                "coef":list(coef[coef["features_group"] == feature].iloc[:,-1]) + [0]})
            max_ = np.max(all_coef["coef"])
            min_ = np.min(all_coef["coef"])
            scaled_coef = []
            for item in all_coef["coef"]:
                scaled_coef.append((item - min_)/(max_ - min_))
            value_dict = dict()
            for i in range(len(all_coef)):
                value_dict[all_coef["level"][i]] = scaled_coef[i]
        feature_dict[feature] = value_dict
    
    brand_score[brand] = feature_dict




In [11]:
# Score for Alpha
brand_score["Bezt"]

{'ad_exp': {"Don't recall seeing an ad": 0.5350588844970363,
  'Some other type of ad': 1.0,
  'Sponsored Brands': 0.5277640227755558,
  'Sponsored Products': 0.5815754107510374,
  'Video ad': 0.4009709936599937,
  'Display/banner ad': 0.0},
 'age': {'25-34': 0.8002892226342878,
  '35-44': 0.9150270465619798,
  '45-54': 0.8925638849940775,
  '55-64': 0.866218119351058,
  '65+': 1.0,
  '18-24': 0.0},
 'income': {'$20,000 - $39,999': 0.581658949652101,
  '$40,000 - $59,999': 1.0,
  '$60,000 - $79,999': 0.19467062400833704,
  '$80,000 - $99,999': 0.16766945884456186,
  'Less than $20,000': 0.5004469171667982,
  '$100,000 or more': 0.0}}

## 4. WOOF function

In [69]:
#Ended up not using Nick's weight
# weight = {'ad_exp': 0.004615311769551043,
#           'age': 0.8243431825876556,
#           'income': 0.17104150564279336}

In [66]:
def WOOF(brand, customer_profile):
    global brands
    global weight_dict
    global brand_score
    
    try: 
        selected_brand_score = brand_score[brand]
    except:
        print("Error! Brand not found")
    
    # just need to access features stored in the brand_score
    selected_features = list(brand_score[brands[5]].keys())
    # slice the customer (1 row of) df
    customer_profile = customer_profile[selected_features]
    
    score = 0
    for feature in selected_features:
                # access the weight from 2.) and scores the from selected brand 
        score += weight_dict[feature]*selected_brand_score[feature][customer_profile[feature][0]]
        
    return score
    

## 5. Business Application

In [67]:
# Customer profile 
cust_profile = df.drop_duplicates("customer_id", keep = "first")
# cust_profile = cust_profile[["customer_id",'sns','ad_exp', 'gender', 'city', 'st',
#        'zip', 'lat', 'lng', 'marital', 'education', 'income', 'age', 'prime']]
print(cust_profile.shape)
cust_profile.head()

(4679, 22)


,sale_id,sale_date,ad_exp,sns,product_id,product_brand,product_name,price,qty,customer_id,...,st,zip,lat,lng,marital,education,income,age,prime,total_amount
0,1,2021-10-01,Don't recall seeing an ad,0,3,Alpha,"Alpha Natural Sensitive Systems, Skin & Coat S...",48.99,1,370,...,ID,83711,43.4599,-116.2440,Married,Some college or trade school,"$80,000 - $99,999",55-64,0,48.99
1,2,2021-10-01,Don't recall seeing an ad,1,14,Arf,Arf Soft & Tender American Jerky Dog Treats,11.52,1,625,...,NC,27710,36.0512,-78.8577,Married,High school graduate,"$100,000 or more",45-54,1,11.52
2,3,2021-10-01,Don't recall seeing an ad,0,23,Bezt,Bezt Adult Chicken and Brown Rice Recipe Dry D...,14.22,1,905,...,AZ,85099,33.2765,-112.1872,Married,College graduate,"$100,000 or more",45-54,1,14.22
3,4,2021-10-01,Don't recall seeing an ad,0,33,Alpha,"Alpha Probiotics Shredded Blend High Protein, ...",79.31,1,1061,...,NH,214,43.0059,-71.0132,Married,Some college or trade school,"$100,000 or more",55-64,1,79.31
4,5,2021-10-01,Don't recall seeing an ad,0,26,Alpha,Alpha Natural Adult Lamb & Rice Dry Dog Food,25.16,1,1411,...,IL,60624,41.8804,-87.7223,Single,Some college or trade school,"$40,000 - $59,999",25-34,1,25.16


In [68]:
# Get top n customers for each brand 
def get_top_n(brand, n, show_score = False):
    global cust_profile
    scores = []
    for idx in list(cust_profile["customer_id"]):
        scores.append(WOOF(brand, cust_profile[cust_profile["customer_id"] == idx].reset_index(drop = True)))
    scores = pd.DataFrame({"customer_id" :cust_profile["customer_id"],
                          "scores" : scores})
    scores = scores.sort_values("scores", ascending = False).reset_index(drop = True).iloc[:n,:]
    
    if show_score == False:
        output = scores["customer_id"].to_list()
    else:
        output = scores
        
    return output

# Alpha

In [118]:
# Calculate the mean of total amount for the top n
brand = 'Alpha'
n = 100
top_n_cust_id = get_top_n(brand, n, show_score = True)['customer_id']
top_n_cust = cust_profile[cust_profile['customer_id'].isin(top_n_cust_id)]
print(f'Check the number of top {n} customers for {brand}: {len(top_n_cust)}')

Check the number of top 100 customers for Alpha: 100


In [119]:
print(f'Mean of total amount for {brand}: {round(top_n_cust["total_amount"].mean(),2)}')

Mean of total amount for Alpha: 41.78


Compare 43.31 with the mean calculated as follows

## Randomly pick up n customers from a specific brand

In [98]:
import random

### From non-duplicated data

Just tried with cust_profile (non-duplicated data) but you can disregard it

In [99]:
brand_cust_id = cust_profile[cust_profile['product_brand'] == brand]['customer_id']
print(f'Num of unique customers for {brand}: {len(brand_cust_id)}')

Num of unique customers for Alpha: 1528


In [100]:
random.seed(123)
n_brand_cust_id = random.sample(list(brand_cust_id), n)
n_brand_cust = cust_profile[cust_profile['customer_id'].isin(n_brand_cust_id)]
print(f'Check the number of selected customers for {brand}: {len(n_brand_cust)}')

Check the number of selected customers for Alpha: 100


In [101]:
print(f'Mean of total amount for {brand}: {round(n_brand_cust["total_amount"].mean(),2)}')

Mean of total amount for Alpha: 52.22


### From original data

Select 300 "unique" customers from the brand but when calculating the mean, it will include several purchases made by the same customer.

In [102]:
brand_cust_id = df[df['product_brand'] == brand]['customer_id'].unique()
print(f'Num of unique customers for {brand}: {len(brand_cust_id)}')

Num of unique customers for Alpha: 2187


In [103]:
random.seed(123)
n_brand_cust_id = random.sample(list(brand_cust_id), n)
n_brand_cust = df[df['customer_id'].isin(n_brand_cust_id)]
print(f'Check the number of selected customers for {brand}: {len(n_brand_cust)}')

Check the number of selected customers for Alpha: 224


In [104]:
print(f'Mean of total amount for {brand}: {round(n_brand_cust["total_amount"].mean(),2)}')

Mean of total amount for Alpha: 44.13


## Randonly pick up n customers from the whole population

### From non-duplicated data

You can also disregard this 

In [120]:
random.seed(123)
cust_id = list(cust_profile['customer_id'])
n_cust_id = random.sample(cust_id, n)
n_cust = cust_profile[cust_profile['customer_id'].isin(n_cust_id)]
print(f'Check the number of selected customers from the whole population: {len(n_cust)}')
print(f'Mean of total amount: {round(n_cust["total_amount"].mean(),2)}')

Check the number of selected customers from the whole population: 100
Mean of total amount: 48.1


### From original data

Select 300 "unique" customers from the whole customers but when calculating the mean, it will include several purchases made by the same customer.

In [121]:
random.seed(123)
cust_id = list(df['customer_id'].unique())
n_cust_id = random.sample(cust_id, n)
n_cust = df[df['customer_id'].isin(n_cust_id)]
print(f'Check the number of selected customers from the whole population: {len(n_cust)}')
print(f'Mean of total amount: {round(n_cust["total_amount"].mean(),2)}')

Check the number of selected customers from the whole population: 200
Mean of total amount: 46.24


Since the mean for top 300 based on WOOF was 43.31, our function works better than random selection.

# Arf

In [115]:
# Calculate the mean of total amount for the top n
brand = 'Arf'
n = 100
top_n_cust_id = get_top_n(brand, n, show_score = True)['customer_id']
top_n_cust = cust_profile[cust_profile['customer_id'].isin(top_n_cust_id)]
print(f'Check the number of top {n} customers for {brand}: {len(top_n_cust)}')

Check the number of top 100 customers for Arf: 100


In [86]:
print(f'Mean of total amount for {brand}: {round(top_n_cust["total_amount"].mean(),2)}')

Mean of total amount for Arf: 41.48


## Randomly pick up n customers from a specific brand

In [87]:
import random

### From non-duplicated data

Just tried with cust_profile (non-duplicated data) but you can disregard it

In [88]:
brand_cust_id = cust_profile[cust_profile['product_brand'] == brand]['customer_id']
print(f'Num of unique customers for {brand}: {len(brand_cust_id)}')

Num of unique customers for Arf: 467


In [89]:
random.seed(123)
n_brand_cust_id = random.sample(list(brand_cust_id), n)
n_brand_cust = cust_profile[cust_profile['customer_id'].isin(n_brand_cust_id)]
print(f'Check the number of selected customers for {brand}: {len(n_brand_cust)}')

Check the number of selected customers for Arf: 100


In [90]:
print(f'Mean of total amount for {brand}: {round(n_brand_cust["total_amount"].mean(),2)}')

Mean of total amount for Arf: 27.53


### From original data

Select 300 "unique" customers from the brand but when calculating the mean, it will include several purchases made by the same customer.

In [91]:
brand_cust_id = df[df['product_brand'] == brand]['customer_id'].unique()
print(f'Num of unique customers for {brand}: {len(brand_cust_id)}')

Num of unique customers for Arf: 809


In [92]:
random.seed(123)
n_brand_cust_id = random.sample(list(brand_cust_id), n)
n_brand_cust = df[df['customer_id'].isin(n_brand_cust_id)]
print(f'Check the number of selected customers for {brand}: {len(n_brand_cust)}')

Check the number of selected customers for Arf: 216


In [93]:
print(f'Mean of total amount for {brand}: {round(n_brand_cust["total_amount"].mean(),2)}')

Mean of total amount for Arf: 33.38


## Randonly pick up n customers from the whole population

### From non-duplicated data

You can also disregard this 

In [116]:
random.seed(123)
cust_id = list(cust_profile['customer_id'])
n_cust_id = random.sample(cust_id, n)
n_cust = cust_profile[cust_profile['customer_id'].isin(n_cust_id)]
print(f'Check the number of selected customers from the whole population: {len(n_cust)}')
print(f'Mean of total amount: {round(n_cust["total_amount"].mean(),2)}')

Check the number of selected customers from the whole population: 100
Mean of total amount: 48.1


### From original data

Select 300 "unique" customers from the whole customers but when calculating the mean, it will include several purchases made by the same customer.

In [117]:
random.seed(123)
cust_id = list(df['customer_id'].unique())
n_cust_id = random.sample(cust_id, n)
n_cust = df[df['customer_id'].isin(n_cust_id)]
print(f'Check the number of selected customers from the whole population: {len(n_cust)}')
print(f'Mean of total amount: {round(n_cust["total_amount"].mean(),2)}')

Check the number of selected customers from the whole population: 200
Mean of total amount: 46.24


Since the mean for top 300 based on WOOF was 43.31, our function works better than random selection.

# Bezt

In [122]:
# Calculate the mean of total amount for the top n
brand = 'Bezt'
n = 100
top_n_cust_id = get_top_n(brand, n, show_score = True)['customer_id']
top_n_cust = cust_profile[cust_profile['customer_id'].isin(top_n_cust_id)]
print(f'Check the number of top {n} customers for {brand}: {len(top_n_cust)}')

Check the number of top 100 customers for Bezt: 100


In [123]:
print(f'Mean of total amount for {brand}: {round(top_n_cust["total_amount"].mean(),2)}')

Mean of total amount for Bezt: 47.18


Compare 43.31 with the mean calculated as follows

## Randomly pick up n customers from a specific brand

In [124]:
import random

### From non-duplicated data

Just tried with cust_profile (non-duplicated data) but you can disregard it

In [125]:
brand_cust_id = cust_profile[cust_profile['product_brand'] == brand]['customer_id']
print(f'Num of unique customers for {brand}: {len(brand_cust_id)}')

Num of unique customers for Bezt: 770


In [126]:
random.seed(123)
n_brand_cust_id = random.sample(list(brand_cust_id), n)
n_brand_cust = cust_profile[cust_profile['customer_id'].isin(n_brand_cust_id)]
print(f'Check the number of selected customers for {brand}: {len(n_brand_cust)}')

Check the number of selected customers for Bezt: 100


In [127]:
print(f'Mean of total amount for {brand}: {round(n_brand_cust["total_amount"].mean(),2)}')

Mean of total amount for Bezt: 37.63


### From original data

Select 300 "unique" customers from the brand but when calculating the mean, it will include several purchases made by the same customer.

In [128]:
brand_cust_id = df[df['product_brand'] == brand]['customer_id'].unique()
print(f'Num of unique customers for {brand}: {len(brand_cust_id)}')

Num of unique customers for Bezt: 1309


In [129]:
random.seed(123)
n_brand_cust_id = random.sample(list(brand_cust_id), n)
n_brand_cust = df[df['customer_id'].isin(n_brand_cust_id)]
print(f'Check the number of selected customers for {brand}: {len(n_brand_cust)}')

Check the number of selected customers for Bezt: 242


In [130]:
print(f'Mean of total amount for {brand}: {round(n_brand_cust["total_amount"].mean(),2)}')

Mean of total amount for Bezt: 43.7


## Randonly pick up n customers from the whole population

### From non-duplicated data

You can also disregard this 

In [52]:
random.seed(123)
cust_id = list(cust_profile['customer_id'])
n_cust_id = random.sample(cust_id, n)
n_cust = cust_profile[cust_profile['customer_id'].isin(n_cust_id)]
print(f'Check the number of selected customers from the whole population: {len(n_cust)}')
print(f'Mean of total amount: {round(n_cust["total_amount"].mean(),2)}')

Check the number of selected customers from the whole population: 300
Mean of total amount: 43.33


### From original data

Select 300 "unique" customers from the whole customers but when calculating the mean, it will include several purchases made by the same customer.

In [53]:
random.seed(123)
cust_id = list(df['customer_id'].unique())
n_cust_id = random.sample(cust_id, n)
n_cust = df[df['customer_id'].isin(n_cust_id)]
print(f'Check the number of selected customers from the whole population: {len(n_cust)}')
print(f'Mean of total amount: {round(n_cust["total_amount"].mean(),2)}')

Check the number of selected customers from the whole population: 594
Mean of total amount: 42.6


## Persona

In [12]:
# filter df to only Alpha

df_alpha = df[df['product_brand'] == 'Bezt']

# sample 300 Alpha existing customers
df_alpha_sample = df_alpha.sample(n=300)
df_alpha_sample = df_alpha_sample[['customer_id','ad_exp', 'age', 'income', 'total_amount']]

# find mean of total_amount out of 300 randomly selected Alpha customers
df_alpha_sample_mean = df_alpha_sample['total_amount'].mean()

print('Alpha existing customers has an average of total sales amount of: ', df_alpha_sample_mean)

Alpha existing customers has an average of total sales amount of:  43.288199999999925
